In [ ]:
#import required packages
import pandas as pd
import requests
import pdfkit
import os
import numpy as np
import csv
import numpy as np
from bs4 import BeautifulSoup
import time

In [3]:
# URL of the main catalog page
base_url = "https://records.ureg.virginia.edu/"
main_url = base_url + "content.php?catoid=61&navoid=5303"

# Function to fetch HTML content
def get_soup(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Failed to fetch {url}")
        return None

# Step 1: Get all major links
soup = get_soup(main_url)
major_links = []

if soup:
    table = soup.find("table", class_="table_default")
    if table:
        for row in table.find_all("tr")[1:]:  # Skip header row
            link_tag = row.find("a")  # Find first column link
            if link_tag:
                major_name = link_tag.text.strip()
                major_url = base_url + link_tag["href"].replace("&amp;", "&")
                major_links.append((major_name, major_url))

# Step 2: Visit each major page and extract data
data = []

for major_name, major_url in major_links:
    print(f"Scraping: {major_name}")
    soup = get_soup(major_url)
    
    if soup:
        table = soup.find("table", class_="table_default")
        
        if table:
            text = " ".join(table.stripped_strings)  # Extract all text
            data.append({"Major": major_name, "Text": text})
    
    time.sleep(1)  # Politeness delay

# Step 3: Save data to CSV
df = pd.DataFrame(data)
df.to_csv("uva_majors.csv", index=False)

print("Scraping complete! Data saved to 'uva_majors.csv'.")

Scraping: Advanced Search
Scraping: Advanced Search
Scraping: Advanced Search
Scraping: Catalog Home
Scraping: Catalog Home
Scraping: HELP
Scraping: HELP
Scraping: Architectural History, B.Ar.H.
Scraping: Architectural History, B.Ar.H.
Scraping: Architecture, B.S.
Scraping: Urban and Environmental Planning, B.U.E.P.
Scraping: African-American and African Studies, B.A.
Scraping: Anthropology, B.A.
Scraping: Applied Statistics, B.A.
Scraping: Area Studies, B.A.
Scraping: Art History
Scraping: Studio Art
Scraping: Astronomy, B.A.
Scraping: Astronomy, B.S.
Scraping: Behavioral Neuroscience, B.S.
Scraping: Biology, B.A.
Scraping: Biology, B.S.
Scraping: Chemistry, B.A.
Scraping: Chemistry, B.S.
Scraping: Classics, B.A.
Scraping: Computer Science, B.A.
Scraping: Drama, B.A.
Scraping: Economics, B.A.
Scraping: English, B.A.
Scraping: Environmental Sciences, B.A.
Scraping: Environmental Sciences, B.S.
Scraping: Foreign Affairs, B.A.
Scraping: French, B.A.
Scraping: German, B.A.
Scraping: Gover

In [4]:
#read in csv
majors = pd.read_csv("uva_majors.csv")

#remove the first 8 rows and last row because they are not needed
majors = majors.iloc[8:-1]

#print the first 5 rows
majors

,Major,Text
8,"Architectural History, B.Ar.H.",Info For Students Alumni Military Affiliated S...
9,"Architecture, B.S.",Info For Students Alumni Military Affiliated S...
10,"Urban and Environmental Planning, B.U.E.P.",Info For Students Alumni Military Affiliated S...
11,"African-American and African Studies, B.A.",Info For Students Alumni Military Affiliated S...
12,"Anthropology, B.A.",Info For Students Alumni Military Affiliated S...
...,...,...
88,"Materials Science and Engineering, B.S.",Info For Students Alumni Military Affiliated S...
89,"Mechanical Engineering, B.S.",Info For Students Alumni Military Affiliated S...
90,"Systems Engineering, B.S.",Info For Students Alumni Military Affiliated S...
91,"Public Policy and Leadership, B.A.",Info For Students Alumni Military Affiliated S...


In [20]:
# remove the extra text from the text column
test = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'
text_to_remove_head = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 12, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025' 
text_to_remove_tail = 'Back to Top | Print-Friendly Page (opens a new window) All catalogs © 2025 University of Virginia. Powered by Modern Campus Catalog™ . .'
#function to remove the text
def remove_text(text):
    if text.startswith(text_to_remove_head):
        text = text[len(text_to_remove_head):]
    if text.startswith(test):
        text = text[len(test):]
    if text.endswith(text_to_remove_tail):
        text = text[:-len(text_to_remove_tail)]
    return text.strip()

#apply the function to the text column
majors['Text'] = majors['Text'].apply(remove_text)

#since some of the majors have extra text that is a bit different, we need to remove it
majors[majors['Major'] == 'Psychology, B.A.']['Text']

majors[50:90]

,Major,Text
58,"Mathematics, B.A.","Mathematics, B.A. Print-Friendly Page (opens a..."
59,"Media Studies, B.A.","Media Studies, B.A. Print-Friendly Page (opens..."
60,"Music, B.A.","Music, B.A. Print-Friendly Page (opens a new w..."
61,"Philosophy, B.A.","Philosophy, B.A. Print-Friendly Page (opens a ..."
62,"Physics, B.A.","Physics, B.A. Print-Friendly Page (opens a new..."
63,"Physics, B.S.","Physics, B.S. Print-Friendly Page (opens a new..."
64,"Psychology, B.A.","Psychology, B.A. Print-Friendly Page (opens a ..."
65,"Religious Studies, B.A.","Religious Studies, B.A. Print-Friendly Page (o..."
66,"Slavic Languages and Literatures, B.A.","Slavic Languages and Literatures, B.A. Print-F..."
67,"Sociology, B.A.","Sociology, B.A. Print-Friendly Page (opens a n..."


In [27]:
# remove extra text from the majors column where majors includes "interdisciplinary"
text_to_remove = 'Interdisciplinary Major - '
def remove_text_majors(text):
    if text.startswith(text_to_remove):
        text = text[len(text_to_remove):]
    return text.strip()

#apply the function to the majors column
majors['Major'] = majors['Major'].apply(remove_text_majors)

In [28]:
majors[:40]

,Major,Text
8,"Architectural History, B.Ar.H.","Architectural History, B.Ar.H. Print-Friendly ..."
9,"Architecture, B.S.","Architecture, B.S. Print-Friendly Page (opens ..."
10,"Urban and Environmental Planning, B.U.E.P.","Urban and Environmental Planning, B.U.E.P. Pri..."
11,"African-American and African Studies, B.A.","African American and African Studies, B.A. Pri..."
12,"Anthropology, B.A.","Anthropology, B.A. Print-Friendly Page (opens ..."
13,"Applied Statistics, B.A.","Applied Statistics, B.A. Print-Friendly Page (..."
14,"Area Studies, B.A.","Area Studies, B.A. Print-Friendly Page (opens ..."
15,Art History,History of Art Print-Friendly Page (opens a ne...
16,Studio Art,Studio Art Print-Friendly Page (opens a new wi...
17,"Astronomy, B.A.","Astronomy, B.A. Print-Friendly Page (opens a n..."


In [8]:
# get school requirements



# Define the main URL
url = "https://records.ureg.virginia.edu/content.php?catoid=61&navoid=5916"

# Send a request to the main page
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all the links for schools
school_links = soup.find_all('a', class_='card-link')

# Loop through each school link
for link in school_links:
    school_url = link.get('href')
    full_url = f"https://records.ureg.virginia.edu/{school_url}"
    
    # Send a request to each school page
    school_response = requests.get(full_url)
    school_soup = BeautifulSoup(school_response.content, "html.parser")
    
    # Find and click on "Academic Rules" link
    academic_rules_link = school_soup.find('a', string='Academic Rules')
    if academic_rules_link:
        academic_rules_url = f"https://records.ureg.virginia.edu/{academic_rules_link.get('href')}"
        
        # Send a request to the Academic Rules page
        academic_response = requests.get(academic_rules_url)
        academic_soup = BeautifulSoup(academic_response.content, "html.parser")
        
        # Find the table and print its content
        table = academic_soup.find('table', {'class': 'table default'})
        if table:
            rows = table.find_all('tr')
            for row in rows:
                columns = row.find_all('td')
                for column in columns:
                    print(column.get_text(strip=True))

In [ ]:
# URL of the main catalog page
base_url = "https://records.ureg.virginia.edu/"
main_url = base_url + "content.php?catoid=61&navoid=5916"

# Function to fetch HTML content
def get_soup(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Failed to fetch {url}")
        return None

# Step 1: Get all school links
soup = get_soup(main_url)
school_links = []

if soup:
    # Find all school links
    grid_items = soup.find_all("div", class_="cgw-grid-item")

    if not grid_items:
        print("no grid items found")
    else:
        for item in grid_items:
            school_name = item.find("h3").text.strip() if item.find("h3") else "unknown"

            link_tag = item.find("a", href=True)
            if link_tag:    
                school_url = link_tag["href"].replace("&amp;", "&")
                school_links.append((school_name, school_url))
                print(f"Found link: {school_name} -> {school_url}")  # Debug print


print(f"Found {len(school_links)} schools.")
for school_name, school_url in school_links:
    print(f"School: {school_name}, URL: {school_url}")


# Step 2: Visit each school page and extract data from the "Academic Rules"
data = []

for school_name, school_url in school_links:
    print(f"Scraping: {school_name}")
    soup = get_soup(school_url)
    
    if soup:
        academic_rules_link = soup.find("a", string="Academic Rules")
        
        if academic_rules_link:
            academic_rules_url = base_url + academic_rules_link.get("href").replace("&amp;", "&")
            print(f"Found Academic Rules link: {academic_rules_url}")  # Debug print        
            academic_soup = get_soup(academic_rules_url)
            if academic_soup:
                table = academic_soup.find("table")
                
                if table:
                    text = " ".join(table.stripped_strings)  # Extract all text
                    data.append({"School": school_name, "Text": text})
    
    time.sleep(1)  # Politeness delay

# Step 3: Save data to CSV and data frame
df = pd.DataFrame(data)
df.to_csv("uva_schools_academic_rules.csv", index=False)

print("Scraping complete! Data saved to 'uva_schools_academic_rules.csv'.")

Found link: School of Architecture -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5306
Found link: College of Arts & Sciences -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5305
Found link: McIntire School of Commerce -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5295
Found link: School of Continuing and Professional Studies -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5310
Found link: School of Data Science -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5281
Found link: School of Education and Human Development -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5307
Found link: School of Engineering and Applied Science -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5308
Found link: Frank Batten School of Leadership and Public Policy -> http://records.ureg.virginia.edu/content.php?catoid=61&navoid=5311
Found link: School of Nursing -> http://records.ureg.virginia

In [29]:
# remove the extra text from the text column
extra_begining_text = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'

def remove_text_requirements(text):
    if text.startswith(extra_begining_text):
        text = text[len(extra_begining_text):]
    return text.strip()

df['Text'] = df['Text'].apply(remove_text_requirements)



In [30]:
df

,School,Text
0,School of Architecture,"School of Architecture: Academic Rules, Regula..."
1,College of Arts & Sciences,"College of Arts & Sciences: Academic Rules, Re..."
2,McIntire School of Commerce,"McIntire School of Commerce: Academic Rules, R..."
3,School of Continuing and Professional Studies,School of Continuing and Professional Studies:...
4,School of Data Science,School of Data Science: Academic Rules Print-F...
5,School of Education and Human Development,School of Education and Human Development: Aca...
6,School of Engineering and Applied Science,School of Engineering and Applied Science: Aca...
7,Frank Batten School of Leadership and Public P...,Frank Batten School of Leadership and Public P...
8,School of Nursing,"School of Nursing: Academic Rules, Regulations..."
